# Modeling - 2

In [42]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.metrics import log_loss

from utils.utils import *
from utils.modeling import *
from features.ohe_dept import *
from features.base_features import *
from preprocess.pipe1 import PreprocessingV1

### Preprocessing Pipeline 1

In [2]:
# Import file
df = import_data("../data-ignore/train.csv")

In [3]:
# OHE the departments
ohe_dept_df, transformer = ohe_dept(df)

ohe_dept_groupby_df = ohe_dept_groupby(ohe_dept_df)
ohe_dept_groupby_df.reset_index(inplace=True)
ohe_dept_groupby_df

,VisitNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191344,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95671,191345,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Goupby VisitNumber
groupby_df = groupby_visitnumber(df)
groupby_df

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline
0,5,999,Friday,1,-1.000000,-1,1,1
1,7,30,Friday,2,1.000000,2,2,2
2,8,26,Friday,20,1.217391,28,6,16
3,9,8,Friday,3,1.000000,3,2,3
4,10,8,Friday,3,1.000000,3,2,3
...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5
95670,191344,22,Sunday,5,1.000000,5,2,3
95671,191345,39,Sunday,13,1.307692,17,8,12
95672,191346,39,Sunday,17,1.000000,17,8,16


In [5]:
# Join the 2 groupby'ed dataframes on the VisitNumber
df = pd.merge(groupby_df, ohe_dept_groupby_df, on="VisitNumber", how="inner")
df

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,5,999,Friday,1,-1.000000,-1,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,30,Friday,2,1.000000,2,2,2,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,26,Friday,20,1.217391,28,6,16,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,9,8,Friday,3,1.000000,3,2,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,8,Friday,3,1.000000,3,2,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191344,22,Sunday,5,1.000000,5,2,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95671,191345,39,Sunday,13,1.307692,17,8,12,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191346,39,Sunday,17,1.000000,17,8,16,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Pre-preprocess
X_train, X_test, y_train, y_test = pre_preprocess(df)

# Preprocessing
preprocessing = PreprocessingV1()
X_train_proc = preprocessing.fit_transform(X_train)
X_test_proc = preprocessing.transform(X_test)

X_train_proc

X_train: (81322, 75), y_train: (81322,), X_test: (14352, 75), y_test: (14352,)


,num__num_unique_upc,num__avg_scancount,num__total_scancount,num__num_unique_dept,num__num_unique_fileline,num__DepartmentDescription_1-HR PHOTO,num__DepartmentDescription_ACCESSORIES,num__DepartmentDescription_AUTOMOTIVE,num__DepartmentDescription_BAKERY,num__DepartmentDescription_BATH AND SHOWER,...,num__DepartmentDescription_TOYS,num__DepartmentDescription_WIRELESS,num__DepartmentDescription_nan,cat__weekday_Friday,cat__weekday_Monday,cat__weekday_Saturday,cat__weekday_Sunday,cat__weekday_Thursday,cat__weekday_Tuesday,cat__weekday_Wednesday
74630,-0.663062,-0.038488,-0.632608,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
61468,-0.663062,-3.064435,-0.827308,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10207,-0.424896,-0.038488,-0.437909,-0.473153,-0.393816,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25328,0.765932,-0.038488,0.535589,1.976168,0.835391,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15352,0.408683,-0.038488,0.243540,0.576556,0.562234,-0.039015,10.708021,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57115,0.051435,0.177651,0.048840,-0.123250,-0.120659,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,0.0,0.0,1.0,0.0
360,-0.663062,1.474485,-0.535258,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
89473,-0.067648,-0.254627,-0.145859,-0.123250,-0.393816,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,1.0,0.0,0.0,0.0,0.0
51195,0.051435,-0.038488,-0.048510,0.226653,0.015920,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [13]:
# Train on 3 models
results_df = train_model(X_train_proc, y_train, X_test_proc, y_test)
results_df


Running Logistic Regression...
Running Random Forest...
Running XGBoost...


,Model,Train Loss,Test Loss
0,LogReg,1.157684,1.145848
1,RForest,1.635823,1.634854
2,XGBoost,0.912533,0.880425


# Predict on Kaggle

In [32]:
# Import data
kaggle_test = pd.read_csv('../data-ignore/test.csv', dtype={'Upc': str})
kaggle_test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,72503389714,1,SHOES,3002.0
1,1,Friday,1707710732,1,DAIRY,1526.0
2,1,Friday,89470001026,1,DAIRY,1431.0
3,1,Friday,88491211470,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2840015224,1,DSD GROCERY,4408.0


In [33]:
# OHE the departments
kaggle_ohe_dept_df, transformer = ohe_dept(kaggle_test)

kaggle_ohe_dept_groupby_df = ohe_dept_groupby(kaggle_ohe_dept_df)
kaggle_ohe_dept_groupby_df.reset_index(inplace=True)
kaggle_ohe_dept_groupby_df

,VisitNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191339,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95671,191340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# Goupby VisitNumber
kaggle_groupby_df = groupby_visitnumber_kaggle(kaggle_test)
kaggle_groupby_df

,VisitNumber,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline
0,1,Friday,4,1.000000,4,3,4
1,2,Friday,4,1.000000,4,3,3
2,3,Friday,1,0.000000,0,1,1
3,4,Friday,1,1.000000,1,1,1
4,6,Friday,1,0.000000,0,1,1
...,...,...,...,...,...,...,...
95669,191338,Sunday,7,1.714286,12,3,6
95670,191339,Sunday,5,1.200000,6,3,5
95671,191340,Sunday,1,2.000000,2,1,1
95672,191341,Sunday,10,1.200000,12,5,10


In [35]:
# Join the 2 groupby'ed dataframes on the VisitNumber
kaggle_df = pd.merge(kaggle_groupby_df, kaggle_ohe_dept_groupby_df, on="VisitNumber", how="inner")
kaggle_df

,VisitNumber,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,1,Friday,4,1.000000,4,3,4,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Friday,4,1.000000,4,3,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Friday,1,0.000000,0,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Friday,1,1.000000,1,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,Friday,1,0.000000,0,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,Sunday,7,1.714286,12,3,6,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191339,Sunday,5,1.200000,6,3,5,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95671,191340,Sunday,1,2.000000,2,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191341,Sunday,10,1.200000,12,5,10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Save the VisitNumber
kg_index = kaggle_df['VisitNumber'].astype(str)
kg_index.head()

0    1
1    2
2    3
3    4
4    6
Name: VisitNumber, dtype: object

In [37]:
# Preprocessing
preprocessing = PreprocessingV1()
preprocessing.fit_transform(X_train)
kaggle_proc = preprocessing.transform(kaggle_df)

kaggle_proc

,num__num_unique_upc,num__avg_scancount,num__total_scancount,num__num_unique_dept,num__num_unique_fileline,num__DepartmentDescription_1-HR PHOTO,num__DepartmentDescription_ACCESSORIES,num__DepartmentDescription_AUTOMOTIVE,num__DepartmentDescription_BAKERY,num__DepartmentDescription_BATH AND SHOWER,...,num__DepartmentDescription_TOYS,num__DepartmentDescription_WIRELESS,num__DepartmentDescription_nan,cat__weekday_Friday,cat__weekday_Monday,cat__weekday_Saturday,cat__weekday_Sunday,cat__weekday_Thursday,cat__weekday_Tuesday,cat__weekday_Wednesday
0,-0.305814,-0.038488,-0.340559,-0.123250,-0.257237,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.305814,-0.038488,-0.340559,-0.123250,-0.393816,-0.039015,-0.069445,-0.134757,2.072897,-0.094712,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.663062,-1.551462,-0.729958,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.663062,-0.038488,-0.632608,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.663062,-1.551462,-0.729958,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,0.051435,1.042207,0.438239,-0.123250,0.015920,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95670,-0.186731,0.264106,-0.145859,-0.123250,-0.120659,-0.039015,-0.069445,2.213364,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95671,-0.663062,1.474485,-0.535258,-0.823057,-0.666973,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95672,0.408683,0.264106,0.438239,0.576556,0.562234,-0.039015,-0.069445,-0.134757,-0.196121,-0.094712,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [38]:
# Train on full train dataset
full_X_train = pd.concat([X_train_proc, X_test_proc])
full_y_train = pd.concat([y_train, y_test])

# Retrain model
xgb_model = XGBClassifier()
xgb_model.fit(full_X_train, full_y_train)
train_proba = xgb_model.predict_proba(full_X_train)

# Check performance on train
train_loss = log_loss(full_y_train, train_proba)
print(f"Train loss: {train_loss:.4f}")

# Predict on kaggle test
kg_proba = xgb_model.predict_proba(kaggle_proc)
kg_proba

Train loss: 0.5726


array([[3.63510007e-05, 4.45741489e-06, 1.19907818e-04, ...,
        6.39056601e-03, 6.71393005e-04, 5.21653192e-03],
       [2.94674046e-05, 1.46096281e-05, 2.96708604e-04, ...,
        8.15023296e-03, 3.13332072e-04, 1.96945630e-02],
       [2.31929607e-06, 8.32775982e-08, 1.53133624e-06, ...,
        3.96560836e-07, 1.60114467e-07, 9.98490691e-01],
       ...,
       [1.16698106e-03, 1.27635230e-05, 1.14053786e-04, ...,
        1.85850968e-05, 2.39100718e-05, 5.62671609e-02],
       [6.81200049e-07, 7.68118980e-07, 3.31185447e-06, ...,
        9.42807551e-03, 1.57250166e-02, 1.51296685e-04],
       [5.46079889e-07, 9.41884863e-08, 4.58602472e-07, ...,
        5.09716047e-04, 2.28662728e-04, 1.17167459e-04]],
      shape=(95674, 38), dtype=float32)

In [39]:
# Import headers from sample_submission
sample_sub = pd.read_csv('../data-ignore/sample_submission.csv')
headers = sample_sub.columns.to_list()
headers

['VisitNumber',
 'TripType_3',
 'TripType_4',
 'TripType_5',
 'TripType_6',
 'TripType_7',
 'TripType_8',
 'TripType_9',
 'TripType_12',
 'TripType_14',
 'TripType_15',
 'TripType_18',
 'TripType_19',
 'TripType_20',
 'TripType_21',
 'TripType_22',
 'TripType_23',
 'TripType_24',
 'TripType_25',
 'TripType_26',
 'TripType_27',
 'TripType_28',
 'TripType_29',
 'TripType_30',
 'TripType_31',
 'TripType_32',
 'TripType_33',
 'TripType_34',
 'TripType_35',
 'TripType_36',
 'TripType_37',
 'TripType_38',
 'TripType_39',
 'TripType_40',
 'TripType_41',
 'TripType_42',
 'TripType_43',
 'TripType_44',
 'TripType_999']

In [43]:
# Create submission dataframe with predictions
submission = pd.DataFrame(np.round(kg_proba, 4), index=kg_index, columns=headers[1:])
submission.reset_index(inplace=True)
submission

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.0000,0.0,0.0001,0.0001,0.0084,0.0191,0.0119,0.0013,0.0,...,0.0002,0.0009,0.4009,0.0601,0.0002,0.0055,0.0083,0.0064,0.0007,0.0052
1,2,0.0000,0.0,0.0003,0.0002,0.0538,0.0211,0.0066,0.0048,0.0,...,0.0005,0.0008,0.0371,0.0971,0.0003,0.0020,0.0041,0.0082,0.0003,0.0197
2,3,0.0000,0.0,0.0000,0.0000,0.0001,0.0006,0.0006,0.0000,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9985
3,4,0.0002,0.0,0.0001,0.0002,0.0059,0.0603,0.8522,0.0000,0.0,...,0.0002,0.0001,0.0005,0.0003,0.0001,0.0001,0.0006,0.0000,0.0000,0.0347
4,6,0.0000,0.0,0.0000,0.0000,0.0001,0.0006,0.0006,0.0000,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,0.0000,0.0,0.0000,0.0002,0.0004,0.0001,0.0000,0.0002,0.0,...,0.0000,0.0001,0.0024,0.0598,0.0011,0.0002,0.0004,0.0010,0.0006,0.0002
95670,191339,0.0000,0.0,0.0000,0.0001,0.0010,0.0005,0.0016,0.0001,0.0,...,0.0002,0.0002,0.0019,0.0241,0.0001,0.0071,0.0526,0.0156,0.0008,0.0047
95671,191340,0.0012,0.0,0.0001,0.0038,0.0039,0.7022,0.1778,0.0001,0.0,...,0.0007,0.0001,0.0003,0.0005,0.0001,0.0000,0.0070,0.0000,0.0000,0.0563
95672,191341,0.0000,0.0,0.0000,0.0000,0.0001,0.0000,0.0000,0.0001,0.0,...,0.0000,0.0000,0.0002,0.0420,0.0015,0.0050,0.0062,0.0094,0.0157,0.0002


In [44]:
# Save submission to csv
submission.to_csv('../data-ignore/submission4.csv', header=True, index=False)

In [45]:
# Kaggle score: private = 0.87181, public = 0.89295
# Kaggle place 373 / 1043 = 36%